# Modeling Collaborative Filtering Recommender System

## Importing Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import joblib
from IPython.display import HTML

In [2]:
df_2 = pd.read_csv('wines_and_ratings.csv', low_memory=False, compression="zstd")
df_2

,RatingID,UserID,Rating,Vintage,WineName,ABV,Body,Acidity,Country,RegionName,...,Harmonize3,Harmonize4,Harmonize5,Harmonize6,Harmonize7,Harmonize8,Harmonize9,Harmonize10,Harmonize11,Harmonize12
0,143,1356810,4.5,1950,Presidential Colheita Port,14.5,Very full bodied,High,Portugal,Porto,...,Hard Cheese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,199,1173759,5.0,1951,Pauillac Premier Grand Cru Classé,13.0,Full bodied,High,France,Pauillac,...,Game Meat,Poultry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,348,1164877,5.0,1952,Pauillac Premier Grand Cru Classé,13.0,Full bodied,High,France,Pauillac,...,Game Meat,Poultry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,374,1207665,5.0,1953,Saint Julien Grand Cru Classé,13.5,Full bodied,High,France,Saint Julien,...,Game Meat,Poultry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,834,1075841,5.0,1955,Saint Julien Grand Cru Classé,14.0,Full bodied,High,France,Saint Julien,...,Game Meat,Poultry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,21013438,1000052,4.5,N.V.,Blanc de Blancs Brut Champagne,12.5,Medium bodied,High,France,Champagne,...,Shellfish,Soft Cheese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149996,21013467,1180844,4.0,N.V.,Brut R de Ruinart Champagne,12.0,Medium bodied,High,France,Champagne,...,Shellfish,Soft Cheese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149997,21013494,1218581,3.5,N.V.,Crémant d Alsace Cuvée Julien Brut,12.5,Light bodied,High,France,Crémant d Alsace,...,Appetizer,Snack,Lean Fish,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149998,21013505,1106198,4.5,N.V.,Blanc de Blancs Brut Champagne,12.5,Medium bodied,High,France,Champagne,...,Shellfish,Soft Cheese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating a Pivot Table to Convert User Ratings to a Wide Format

In [3]:
df_2_wide = pd.pivot_table(df_2, values=['Rating'],
                            index=['WineName', 'UserID'],
                            aggfunc=np.mean).unstack()
df_2_wide = df_2_wide.fillna(0)
df_2_wide

/var/folders/6z/81ycht5s235dy_6ymmclw8p00000gn/T/ipykernel_35444/1785985137.py:1: FutureWarning: The provided callable <function mean at 0x10722e8e0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_2_wide = pd.pivot_table(df_2, values=['Rating'],


Rating                  \
UserID                                          1000004 1000010 1000021   
WineName                                                                  
10 Anos Tawny Porto                                 0.0     0.0     0.0   
10 Year Old Tawny Port                              3.5     0.0     0.0   
10 Year Old Tawny Porto                             0.0     0.0     0.0   
101 Moscato d Asti                                  0.0     0.0     0.0   
12 e Mezzo Primitivo del Salento                    0.0     0.0     0.0   
...                                                 ...     ...     ...   
Zweigelt Classic                                    0.0     0.0     0.0   
Zweigeltrebe                                        0.0     0.0     0.0   
Úrágya Tokaji Furmint                               0.0     0.0     0.0   
Ürziger Würzgarten Riesling Kabinett                0.0     0.0     0.0   
Красностоп Золотовский  Krasnostop Zolotovskiy      0.0     0.0     0.0   

                                                                         \
UserID                                          1000023 1000024 1000025   
WineName                                                                  
10 Anos Tawny Porto                                 0.0     0.0     0.0   
10 Year Old Tawny Port                              0.0     0.0     4.0   
10 Year Old Tawny Porto                             0.0     0.0     0.0   
101 Moscato d Asti                                  0.0     0.0     0.0   
12 e Mezzo Primitivo del Salento                    0.0     0.0     0.0   
...                                                 ...     ...     ...   
Zweigelt Classic                                    0.0     0.0     0.0   
Zweigeltrebe                                        0.0     0.0     0.0   
Úrágya Tokaji Furmint                               0.0     0.0     0.0   
Ürziger Würzgarten Riesling Kabinett                0.0     0.0     0.0   
Красностоп Золотовский  Krasnostop Zolotovskiy      0.0     0.0     0.0   

                                                                         \
UserID                                          1000027 1000029 1000031   
WineName                                                                  
10 Anos Tawny Porto                                 0.0     0.0     0.0   
10 Year Old Tawny Port                              0.0     0.0     0.0   
10 Year Old Tawny Porto                             0.0     0.0     0.0   
101 Moscato d Asti                                  0.0     0.0     0.0   
12 e Mezzo Primitivo del Salento                    0.0     0.0     0.0   
...                                                 ...     ...     ...   
Zweigelt Classic                                    0.0     0.0     0.0   
Zweigeltrebe                                        0.0     0.0     0.0   
Úrágya Tokaji Furmint                               0.0     0.0     0.0   
Ürziger Würzgarten Riesling Kabinett                0.0     0.0     0.0   
Красностоп Золотовский  Krasnostop Zolotovskiy      0.0     0.0     0.0   

                                                         ...                  \
UserID                                          1000036  ... 2058533 2060068   
WineName                                                 ...                   
10 Anos Tawny Porto                                 0.0  ...     0.0     0.0   
10 Year Old Tawny Port                              0.0  ...     0.0     0.0   
10 Year Old Tawny Porto                             0.0  ...     0.0     0.0   
101 Moscato d Asti                                  0.0  ...     0.0     0.0   
12 e Mezzo Primitivo del Salento                    0.0  ...     0.0     0.0   
...                                                 ...  ...     ...     ...   
Zweigelt Classic                                    0.0  ...     0.0     0.0   
Zweigeltrebe                                        0.0  ...     0.0     0.0   
Úrágya Tokaji Fu

## Calculating Cosine Similarity and Euclidean Distance Matrices

In [4]:
cosine_sim2 = cosine_similarity(df_2_wide)
cosine_sim2_df = pd.DataFrame(cosine_sim2, index=df_2_wide.index, columns=df_2_wide.index)
cosine_sim2_df

WineName,10 Anos Tawny Porto,10 Year Old Tawny Port,10 Year Old Tawny Porto,101 Moscato d Asti,12 e Mezzo Primitivo del Salento,20 Anos Tawny Porto,20 Year Old Tawny Port,20 Years Old Tawny Porto,21 Gables Chenin Blanc,A Galet Rosado,...,Winemaker s Collection Blanc de Noir,Winemaker s Collection Montepulciano d Abruzzo,Woodthorpe Vineyard Syrah,Zinfandel,Zweigelt,Zweigelt Classic,Zweigeltrebe,Úrágya Tokaji Furmint,Ürziger Würzgarten Riesling Kabinett,Красностоп Золотовский Krasnostop Zolotovskiy
WineName,,,,,,,,,,,,,,,,,,,,,
10 Anos Tawny Porto,1.000000,0.045213,0.000000,0.000000,0.011653,0.000000,0.021286,0.000000,0.038204,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018372
10 Year Old Tawny Port,0.045213,1.000000,0.025639,0.047145,0.073112,0.010292,0.062177,0.043146,0.061025,0.000000,...,0.000000,0.007823,0.031786,0.01283,0.008708,0.000000,0.018565,0.016206,0.040775,0.028322
10 Year Old Tawny Porto,0.000000,0.025639,1.000000,0.000000,0.006419,0.000000,0.023941,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.061881,0.000000,0.000000,0.000000,0.000000,0.000000
101 Moscato d Asti,0.000000,0.047145,0.000000,1.000000,0.007088,0.000000,0.031213,0.000000,0.039055,0.000000,...,0.000000,0.000000,0.064360,0.00000,0.000000,0.042447,0.000000,0.000000,0.021028,0.020434
12 e Mezzo Primitivo del Salento,0.011653,0.073112,0.006419,0.007088,1.000000,0.012885,0.038477,0.011343,0.033697,0.010544,...,0.016346,0.008747,0.016848,0.00000,0.005451,0.013545,0.000000,0.006284,0.006710,0.031559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zweigelt Classic,0.000000,0.000000,0.000000,0.042447,0.013545,0.000000,0.000000,0.000000,0.041141,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
Zweigeltrebe,0.000000,0.018565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023530,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
Úrágya Tokaji Furmint,0.000000,0.016206,0.000000,0.000000,0.006284,0.000000,0.000000,0.000000,0.031023,0.000000,...,0.000000,0.000000,0.112958,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [5]:
dist_euc = euclidean_distances(df_2_wide)
dist_euc_df = pd.DataFrame(dist_euc, index=df_2_wide.index, columns=df_2_wide.index)
dist_euc_df

WineName,10 Anos Tawny Porto,10 Year Old Tawny Port,10 Year Old Tawny Porto,101 Moscato d Asti,12 e Mezzo Primitivo del Salento,20 Anos Tawny Porto,20 Year Old Tawny Port,20 Years Old Tawny Porto,21 Gables Chenin Blanc,A Galet Rosado,...,Winemaker s Collection Blanc de Noir,Winemaker s Collection Montepulciano d Abruzzo,Woodthorpe Vineyard Syrah,Zinfandel,Zweigelt,Zweigelt Classic,Zweigeltrebe,Úrágya Tokaji Furmint,Ürziger Würzgarten Riesling Kabinett,Красностоп Золотовский Krasnostop Zolotovskiy
WineName,,,,,,,,,,,,,,,,,,,,,
10 Anos Tawny Porto,0.000000,92.834530,19.748418,28.600699,85.486841,22.011361,48.523190,24.949950,56.307193,22.710130,...,19.943671,23.318448,26.734809,26.052831,23.895606,21.534855,18.261982,24.989998,32.714676,36.854443
10 Year Old Tawny Port,92.834530,0.000000,92.513513,93.996011,120.117651,93.154442,100.400448,93.240281,104.147732,93.471921,...,92.838570,93.493315,93.872254,94.085068,93.617573,93.193347,92.340403,93.751000,95.281163,97.085014
10 Year Old Tawny Porto,19.748418,92.513513,0.000000,25.980762,84.764379,18.479719,47.106263,21.897488,55.637218,19.306735,...,15.960890,20.018741,23.911294,23.146274,20.099751,17.909495,13.802174,21.943108,30.450780,35.146124
101 Moscato d Asti,28.600699,93.996011,25.980762,0.000000,87.160771,27.739863,51.029403,30.124741,58.510683,28.297526,...,26.129485,28.788018,30.590031,31.044323,29.257478,26.846788,24.869660,30.157918,36.431442,40.388736
12 e Mezzo Primitivo del Salento,85.486841,120.117651,84.764379,87.160771,0.000000,85.202699,94.315958,85.985464,98.673705,85.409309,...,84.695041,85.593516,86.381422,86.520229,85.802098,85.080844,84.501479,86.090069,88.573416,89.488826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zweigelt Classic,21.534855,93.193347,17.909495,26.846788,85.080844,20.377684,48.137823,23.521267,55.733742,21.130547,...,18.124569,21.783021,25.406692,24.688054,22.399777,0.000000,16.255768,23.563743,31.638584,36.180105
Zweigeltrebe,18.261982,92.340403,13.802174,24.869660,84.501479,16.881943,46.765372,20.566964,54.936327,17.783419,...,14.080128,18.553975,22.699119,21.891779,19.274335,16.255768,0.000000,20.615528,29.508473,34.332929
Úrágya Tokaji Furmint,24.989998,93.751000,21.943108,30.157918,86.090069,24.000000,49.779514,26.720778,57.148928,24.642443,...,22.118996,25.204166,26.753504,27.753378,25.739075,23.563743,20.615528,0.000000,34.084454,38.337319


## Testing the Model

In [6]:
def find_similar_wines_combined(wines, count=10):
    wines_cosine = [wine for wine in wines if wine in cosine_sim2_df.columns]
    wines_summed_cosine = cosine_sim2_df[wines_cosine].apply(lambda row: np.sum(row), axis=1)
    wines_summed_cosine = wines_summed_cosine.sort_values(ascending=False)
    ranked_wines_cosine = wines_summed_cosine.index[wines_summed_cosine.index.isin(wines_cosine) == False]
    ranked_wines_cosine = ranked_wines_cosine.tolist()
    ranked_wines_cosine = ranked_wines_cosine[:count]

    wines_euclidean = [wine for wine in wines if wine in dist_euc_df.columns]
    wines_summed_euclidean = dist_euc_df[wines_euclidean].apply(lambda row: np.sum(row), axis=1)
    wines_summed_euclidean = wines_summed_euclidean.sort_values()
    ranked_wines_euclidean = wines_summed_euclidean.index[wines_summed_euclidean.index.isin(wines_euclidean) == False]
    ranked_wines_euclidean = ranked_wines_euclidean.tolist()
    ranked_wines_euclidean = ranked_wines_euclidean[:count]

    combined_ranking = {}
    for i, wine in enumerate(ranked_wines_cosine):
        combined_ranking[wine] = combined_ranking.get(wine, 0) + i

    for i, wine in enumerate(ranked_wines_euclidean):
        combined_ranking[wine] = combined_ranking.get(wine, 0) + i

    sorted_combined_ranking = sorted(combined_ranking.items(), key=lambda x: x[1])

    top_10_wine_indices = [df_2.index[df_2['WineName'] == wine].tolist()[0] for wine, _ in sorted_combined_ranking[:count]]
    top_10_wines = df_2.loc[top_10_wine_indices, ['WineName', 'Website']].copy()

    top_10_wines['Website'] = top_10_wines['Website'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')

    display(HTML(top_10_wines.to_html(escape=False, index=False)))

In [7]:
find_similar_wines_combined(['Vecchia Grappa Moscato'], 10)

WineName,Website
STORIA Grappa Merlot,casavalduga.com.br
Cave Cabernet Sauvignon,luizargenta.com.br
Garnacha Rosé,tesco.com
A Rosé Is A Rosé Is A Rosé,luigibosca.com.ar
Grüner Veltliner Alte Reben,urbanihof.at
Perlé Nero,cantineferrari.it
Fragolino,bautistamarti.com
Reserva Chardonnay,fabremontmayou.com
Trollinger Lemberger Trocken,rolf-willy.de
Colheita Tardia Malvasia Bianca,vinicolaaurora.com.br


## Exporting the Model

In [8]:
joblib.dump((cosine_sim2_df, dist_euc_df, df_2), 'collaborative_recommender_model.pkl', compress=('zlib', 3))

['collaborative_recommender_model.pkl']